In [1]:
import arcpy

def get_overlap_list(
    input_fc,                # primary feature class
    overlap_fc,              # feature class to intersect with
    select_field=None,        # field to filter overlap_fc (optional)
    criteria=None,            # filter value(s) (optional)
    output_list_name="overlap_list",  # output list variable name
    id_field="APN"            # field from input_fc to return
):
    """
    Returns a list of values (default APN) from input_fc that overlap with overlap_fc,
    with optional filtering of overlap_fc.
    """
    
    # Make a temp layer for input_fc
    input_layer = "input_layer"
    arcpy.MakeFeatureLayer_management(input_fc, input_layer)

    # If selection criteria is provided, filter overlap_fc
    if select_field and criteria is not None:
        overlap_layer = "overlap_layer"
        arcpy.MakeFeatureLayer_management(overlap_fc, overlap_layer)
        
        if isinstance(criteria, (list, tuple)):
            sql = f"{select_field} IN ({','.join([repr(c) for c in criteria])})"
        else:
            sql = f"{select_field} = '{criteria}'"
        
        arcpy.SelectLayerByAttribute_management(overlap_layer, "NEW_SELECTION", sql)
    else:
        # No filter, use full overlap_fc
        overlap_layer = overlap_fc

    # Select features in input_fc that intersect with overlap_layer
    arcpy.SelectLayerByLocation_management(input_layer, "INTERSECT", overlap_layer)

    # Convert selection to Python list using the requested field
    result_list = [row[0] for row in arcpy.da.SearchCursor(input_layer, [id_field])]

    # Assign to globals (optional, so you can access by name outside function)
    globals()[output_list_name] = result_list

    return result_list


In [ ]:
import os
arcpy.env.overwriteOutput = True
# set workspace to your geodatabase
gdb = r"F:\GIS\DB_CONNECT\Vector.sde"
# Input feature classes


arcpy.env.overwriteOutput = True


regional_fc = os.path.join(gdb, r"Planning\RegionalLandUse")


apn_list = get_overlap_list(
    overlap_fc=regional_fc,
    parcel_fc=parcel_fc,
    select_field="Description",
    criteria=["Recreation", "Conservation"],
    apn_field="APN",
    out_name="Rec_Con_Parcels"
)

print(apn_list)

['007-011-01', '007-011-05', '007-011-06', '007-011-09', '007-011-10', '007-011-12', '007-011-13', '007-011-14', '007-011-15', '007-011-16', '007-011-17', '007-011-18', '007-011-19', '007-011-20', '007-011-22', '007-011-23', '007-011-24', '007-011-25', '007-011-26', '007-011-27', '007-011-28', '007-011-29', '007-011-30', '007-011-31', '007-021-01', '007-021-02', '007-021-03', '007-021-05', '007-021-06', '007-021-07', '007-021-08', '007-021-09', '007-021-11', '007-021-12', '007-021-14', '007-021-15', '007-021-16', '007-021-17', '007-021-18', '007-021-19', '007-021-20', '007-031-01', '007-031-02', '007-031-13', '007-031-14', '007-031-16', '007-031-17', '010-170-001', '010-170-002', '010-180-004', '010-190-001', '010-190-002', '010-190-003', '010-190-005', '010-190-006', '014-021-001', '014-021-002', '014-021-006', '014-021-008', '014-021-009', '014-021-010', '014-031-001', '014-031-002', '014-031-004', '014-031-009', '014-031-010', '014-031-012', '014-031-015', '014-031-016', '014-040-00

In [ ]:
#Use this list to update parcels in collection/parcel_permit_review_attributes

In [2]:
import os
arcpy.env.overwriteOutput = True
# set workspace to your geodatabase
gdb = r"F:\GIS\DB_CONNECT\Vector.sde"
# Input feature classes


arcpy.env.overwriteOutput = True

In [5]:
# Define overlaps as a list of dictionaries
overlap_list = [
    {
        "overlap_fc": r"Planning\RegionalLandUse",
        "select_field": "Description",
        "criteria": ["Recreation", "Conservation"],
        "apn_field": "APN",
        "out_name": "Rec_Con_Parcels"
    },
    {
        "overlap_fc": r"Historic/HistoricDistricts",
        "select_field": None,
        "criteria": None,
        "apn_field": "APN",
        "out_name": "Historic_Parcels"
    },
    {
        "overlap_fc": r"Water/FEMA_Flood_Zone",
        "select_field": "FLOOD_YEAR",
        "criteria": ["100-Year flood"],
        "apn_field": "APN",
        "out_name": "Flood_Parcels"
    },
    {
        "overlap_fc": r"Scenic/Scenic_Corridor",
        "select_field": None,
        "criteria": None,
        "apn_field": "APN",
        "out_name": "scenic_parcels"
    },
    {
        "overlap_fc": r"Scenic/Scenic_Corridor",
        "select_field": "CATEGORY",
        "criteria": ["Shoreline"],
        "apn_field": "APN",
        "out_name": "scenic_parcels_shoreline"
    },
    {
        "overlap_fc": r"Scenic/Scenic_Corridor",
        "select_field": "CATEGORY",
        "criteria": ["Roadway"],
        "apn_field": "APN",
        "out_name": "scenic_parcels_roadway"
    }
]

# loop through the list and get the apn list for each overlap feature class
apn_lists = {}

parcel_fc = os.path.join(gdb, r"Parcels\Parcel_Master")

for params in overlap_list:
    apn_list = get_overlap_list(
        overlap_fc=os.path.join(gdb, params["overlap_fc"]),
        input_fc=parcel_fc,
        select_field=params["select_field"],
        criteria=params["criteria"],
        output_list_name=params["out_name"]
    )
    apn_lists[params['out_name']] = apn_list

    print(f"{params['out_name']}: {apn_list}")


Rec_Con_Parcels: ['125-442-02', '130-331-02', '1318-15-401-001', '014-322-021', '032-050-066', '025-801-004', '034-084-014', '020-130-006', '025-010-055', '083-440-004', '122-116-11', '125-433-03', '016-531-007', '029-320-001', '033-652-002', '036-110-012', '097-091-001', '125-156-09', '1418-11-111-001', '1318-22-001-010', '1318-23-611-015', '1318-15-611-004', '027-344-033', '023-691-011', '020-130-013', '091-140-011', '090-151-028', '110-051-039', '025-405-007', '032-291-034', '080-161-005', '081-121-005', '035-100-004', '018-071-002', '112-130-023', '090-041-023', '112-120-071', '111-190-021', '123-102-07', '125-780-00', '110-051-042', '125-391-05', '1318-26-601-008', '025-010-051', '032-351-001', '097-050-051', '093-600-002', '092-010-033', '116-140-001', '112-160-024', '125-172-28', '1318-23-510-014', '031-331-020', '025-621-004', '025-757-009', '025-291-001', '032-130-009', '026-154-017', '031-103-013', '032-354-009', '033-781-006', '1319-18-413-007', '017-041-009', '032-120-018',

In [6]:
import arcpy

def update_fc_with_apn_lists(fc, apn_lists, apn_field="APN"):
    """
    Updates a feature class so that for each list in apn_lists, 
    the corresponding field is set to 'Yes' if APN is in the list, 'No' otherwise.
    Works with SDE data by using an edit session.
    """
    workspace = arcpy.Describe(fc).path
    editor = arcpy.da.Editor(workspace)

    # Ensure all target fields exist in the feature class
    existing_fields = [f.name for f in arcpy.ListFields(fc)]
    for field in apn_lists.keys():
        if field not in existing_fields:
            arcpy.AddField_management(fc, field, "TEXT", field_length=3)

    # Start edit session
    editor.startEditing(False, True)
    editor.startOperation()

    try:
        fields = [apn_field] + list(apn_lists.keys())
        with arcpy.da.UpdateCursor(fc, fields) as cursor:
            for row in cursor:
                apn = row[0]
                # Update each field based on membership in the list
                for i, field in enumerate(apn_lists.keys(), start=1):
                    row[i] = "Yes" if apn in apn_lists[field] else "No"
                cursor.updateRow(row)

        # Stop operation and save
        editor.stopOperation()
        editor.stopEditing(True)  # True = save changes
        print("Update complete")
    except Exception as e:
        # Rollback if failure
        editor.stopOperation()
        editor.stopEditing(False)
        raise e



In [7]:
mapping = {
    "Rec_Con_Parcels": "CONSERVATION_REC",
    "Historic_Parcels": "HISTORIC_DISTRICT",
    "Flood_Parcels": "FLOOD_ZONE",
    "scenic_parcels": "SCENIC_CORRIDOR",
    "scenic_parcels_shoreline": "SCENIC_CORRIDOR_SHORELINE",
    "scenic_parcels_roadway": "SCENIC_CORRIDOR_RDWY"
}
apn_lists = {mapping[k]: v for k, v in apn_lists.items()}
gdb = r"F:\GIS\DB_CONNECT\Collection.sde"
parcel_fc = os.path.join(gdb, r"Parcel_Permit_Review_Attributes")

In [8]:
update_fc_with_apn_lists(parcel_fc, apn_lists, apn_field="APN")

Update complete


In [14]:
import arcpy, csv, os

def wkt_csv_to_polyline_fc(
    csv_path,
    out_fc,                         # e.g., os.path.join(gdb, "Coverage_Lines")
    wkt_field=None,                 # if None, auto-detects column containing 'LINESTRING'
    spatial_ref=4326,               # WKID or arcpy.SpatialReference; default WGS84
    overwrite=True
):
    """
    Convert a CSV with WKT LINESTRING/MULTILINESTRING geometry into a polyline feature class.

    - Copies all non-WKT columns as TEXT fields (length 255).
    - Uses arcpy.FromWKT(...) to build geometry.
    """
    if overwrite:
        arcpy.env.overwriteOutput = True

    # Spatial reference
    sr = spatial_ref if isinstance(spatial_ref, arcpy.SpatialReference) else arcpy.SpatialReference(spatial_ref)

    # Read header + small sample to auto-detect WKT column if needed
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        headers = reader.fieldnames

        if not headers:
            raise ValueError("CSV appears to have no header row.")

        # Auto-detect WKT field if not provided
        if wkt_field is None:
            candidate = None
            for _ in range(100):  # peek up to 100 rows
                try:
                    row = next(reader)
                except StopIteration:
                    break
                for h in headers:
                    val = (row.get(h) or "").upper()
                    if "LINESTRING" in val:  # match LINESTRING or MULTILINESTRING
                        candidate = h
                        break
                if candidate:
                    break
            if not candidate:
                raise ValueError("Could not find a WKT field containing 'LINESTRING' in the first 100 rows.")
            wkt_field = candidate

    # Re-open reader to process from the top
    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        headers = reader.fieldnames

        # Attribute fields = everything except the WKT field
        attr_fields = [h for h in headers if h != wkt_field]

        # Create the output feature class
        out_path, out_name = os.path.split(out_fc)
        if not arcpy.Exists(out_path):
            raise ValueError(f"Output workspace does not exist: {out_path}")

        if arcpy.Exists(out_fc) and overwrite:
            arcpy.management.Delete(out_fc)

        arcpy.management.CreateFeatureclass(
            out_path=out_path,
            out_name=out_name,
            geometry_type="POLYLINE",
            spatial_reference=sr
        )

        # Add attribute fields (as TEXT)
        existing_names = {f.name.upper(): f for f in arcpy.ListFields(out_fc)}
        for h in attr_fields:
            valid_name = arcpy.ValidateFieldName(h, out_path)
            # Avoid duplicates after validation
            if valid_name.upper() in existing_names:
                continue
            arcpy.management.AddField(out_fc, valid_name, "TEXT", field_length=255)

        # Final field order for cursor (validated names)
        fc_fields = [arcpy.ValidateFieldName(h, out_path) for h in attr_fields] + ["SHAPE@"]

        # Insert rows
        inserted = 0
        skipped = 0
        with arcpy.da.InsertCursor(out_fc, fc_fields) as icur:
            for row in reader:
                wkt = row.get(wkt_field)
                if not wkt or "LINESTRING" not in wkt.upper():
                    skipped += 1
                    continue
                try:
                    geom = arcpy.FromWKT(wkt, sr)
                    # Build attribute tuple (ensure order matches fc_fields)
                    values = [row.get(h, None) for h in attr_fields] + [geom]
                    icur.insertRow(values)
                    inserted += 1
                except Exception as ex:
                    # Skip malformed WKT rows
                    skipped += 1

    print(f"Created: {out_fc}")
    print(f"Inserted {inserted} feature(s), skipped {skipped} row(s).")


# ===== Example call =====
# Assumes you already set: gdb = r"C:\path\to\your.gdb"
# csv_path = r"/mnt/data/TRPA Coverage_Jan 20240121-28 - Sheet1.csv"
# out_fc   = os.path.join(gdb, "TRPA_Coverage_Lines")

# wkt_csv_to_polyline_fc(csv_path, out_fc, wkt_field=None, spatial_ref=4326, overwrite=True)
csv_path = r"C:\Users\amcclary\Downloads\TRPA Coverage_Jan 20240121-28 - Sheet1.csv"

gdb = r"F:\GIS\PROJECTS\Transportation\Replica\Replica.gdb"
out_fc = os.path.join(gdb, "TRPA_Coverage_Lines_Jan_20240121_28")
wkt_csv_to_polyline_fc(csv_path, out_fc, wkt_field=None, spatial_ref=4326, overwrite=True)

Created: F:\GIS\PROJECTS\Transportation\Replica\Replica.gdb\TRPA_Coverage_Lines_Jan_20240121_28
Inserted 79652 feature(s), skipped 0 row(s).


In [15]:
csv_path = r"C:\Users\amcclary\Downloads\TRPA Coverage_July 20240714-21 - Sheet1.csv"
out_fc = os.path.join(gdb, "TRPA_Coverage_Lines_July_20240714_21")
wkt_csv_to_polyline_fc(csv_path, out_fc, wkt_field=None, spatial_ref=4326, overwrite=True)

Created: F:\GIS\PROJECTS\Transportation\Replica\Replica.gdb\TRPA_Coverage_Lines_July_20240714_21
Inserted 70522 feature(s), skipped 0 row(s).
